In [119]:
import nltk
import random
import os
import json
from nltk.stem import  PorterStemmer
import random
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn import svm
import numpy as np
from collections import Counter
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

In [118]:
label_map={'neg':0 ,'sli_neg':1 ,'neutral':2 ,'sli_pos':3 ,'pos':4}

In [2]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset_new'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,each_file) for sentence in data_temp]
    return dataset

In [4]:
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)

In [7]:
def tags(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [94]:
def get_polarity(word,tag):
    #print 'Get Polarity'
    #print 'Word : '+str(word)+' Tag : '+str(tag)
    synset = swn.senti_synsets(word,tag)
    if(len(synset)>0):
        pos_pol=synset[0].pos_score()
        neg_pol=synset[0].neg_score()
        #print str(pos_pol)+' '+str(neg_pol)
        #print ' '    
        if(pos_pol == neg_pol):
            return 0
        if(pos_pol>neg_pol):
            return pos_pol
        else:
            return (-neg_pol)
    return 0   

In [72]:
def frange(start, end, step):
    tmp = start
    while(tmp < end):
        yield tmp
        tmp += step  

In [121]:
def extract_features(sentence):       
    features =Counter()
    for i in frange(-1,1.05,0.05):
        features[round(i,2)]=0
    pos=nltk.pos_tag(sentence.split())
    for word,tag in pos:
        tag=tags(tag)
        polarity=get_polarity(word,tag)
        features[round(polarity,2)] += 1
    feature_vector=[]                   # feature vector
    for i in frange(-1,1.05,0.05):
        feature_vector.append(features[round(i,1)])    
    return feature_vector

In [109]:
extract_features('this is a very good company')

Counter({-1.0: 0,
         -0.95: 0,
         -0.9: 0,
         -0.85: 0,
         -0.8: 0,
         -0.75: 0,
         -0.7: 0,
         -0.65: 0,
         -0.6: 0,
         -0.55: 0,
         -0.5: 0,
         -0.45: 0,
         -0.4: 0,
         -0.35: 0,
         -0.3: 0,
         -0.25: 0,
         -0.2: 0,
         -0.15: 0,
         -0.1: 0,
         -0.05: 0,
         0.0: 4,
         0.05: 0,
         0.1: 0,
         0.15: 0,
         0.2: 0,
         0.25: 1,
         0.3: 0,
         0.35: 0,
         0.4: 0,
         0.45: 0,
         0.5: 0,
         0.55: 0,
         0.6: 0,
         0.65: 0,
         0.7: 0,
         0.75: 1,
         0.8: 0,
         0.85: 0,
         0.9: 0,
         0.95: 0,
         1.0: 0})

In [123]:
train_test = nltk.classify.apply_features(extract_features, labeled_data)

In [124]:
#dividing into training and test data
train_set=train_test[:-10000]
test_set=train_test[-10000:]


In [125]:
print train_set[1]

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'neutral')


In [113]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
with open('senti_naive_bayes_unigram_model','w') as f:
    pickle.dump(classifier,f)

In [114]:
accuracy=nltk.classify.accuracy(classifier, test_set)

In [115]:
accuracy

0.6083

In [126]:
#label=[]
#feature_svm=[]
#for feature,polarity in train_set:
#    label.append(polarity)
#    feature_svm.append(feature)

In [128]:
#vectorizer = CountVectorizer()
#features=vectorizer.fit_transform(feature_svm)